In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2.potentials import patchyProteinMAPK
from msmrd2.integrators import integratorMAPK 

In [ ]:
# Define arbitrarily large list of patchy particles
np.random.seed(seed=1) # seed 1 good used for pentameric ring formation 
numparticles = 3
boxsize = 4
D = 1.0
Drot = 1.0
pyPartlist = [] 

for i in range(numparticles):    
    overlap = True
    while overlap:
        position = np.array([boxsize*np.random.rand()-0.5*boxsize,
                             boxsize*np.random.rand()-0.5*boxsize,
                             boxsize*np.random.rand()-0.5*boxsize])
        overlap = False     
        for j in range(len(pyPartlist)):
            if np.linalg.norm(position - pyPartlist[j].position) < 2.0:
                overlap = True
                continue
                
    orientation = np.array([np.random.rand(),np.random.rand(),np.random.rand(),np.random.rand()])
    orientation = orientation/np.linalg.norm(orientation)
    part = msmrd2.particle(D, Drot, position, orientation)
    part.setType(i)
    part.setState(0)
    pyPartlist.append(part)

In [ ]:
# alternative fixed position set up
position1 = np.array([0.0, 0.0, 0.0]);
position2 = np.array([np.cos(np.pi/4), np.sin(-np.pi/4), 0]);
position3 = np.array([-2, 0, 0]);
orientation1 = np.array([1.0, 0.0, 0.0, 0.0]);
orientation2 = np.array([np.cos(np.pi/2), 0.0, 0.0, np.sin(np.pi/2)]);
orientation3 = np.array([1.0, 0.0, 0.0, 0.0]);
part1 = msmrd2.particle(0,0,D, Drot, position1, orientation1)
part2 = msmrd2.particle(1,0,D, Drot, position2, orientation2)
part3 = msmrd2.particle(2,0,D, Drot, position3, orientation3)
pyPartlist = [part1, part2, part3] 

In [ ]:
# Create list of particles that can be read from msmrd
# note the particles in this list will be independent from the python list.
partlist = msmrd2.integrators.particleList(pyPartlist)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.00001 #0.000005
seed = -1 # seed = -1 uses random device as seed
bodytype = 'rigidbody' # three rotational degrees of freedom
anglePatches = np.pi/2
reactivationRateK = 10
reactivationRateP = 10
mapkIndex = [0]
kinaseIndex = [1]
phosIndex = [2]
integrator = integratorMAPK(dt, seed, bodytype, anglePatches, reactivationRateK, reactivationRateP, 
                            mapkIndex, kinaseIndex, phosIndex) 
#integrator.setKbT(0.1)

In [ ]:
# Define boundary (choose either spherical or box)
boxBoundary = msmrd2.box(boxsize,boxsize,boxsize,'reflective')
integrator.setBoundary(boxBoundary)

In [ ]:
# Define MAPK potntial based on patchy particles
sigma = 1.0
strength = 60 #65
# This values are fixed and should match those used to determine metastable states in potential and trajectory.
patchesCoordinates1 = [np.array([np.cos(anglePatches/2), np.sin(anglePatches/2), 0.]), 
                       np.array([np.cos(-anglePatches/2), np.sin(-anglePatches/2), 0.])]
patchesCoordinates2 = [ np.array([np.cos(-anglePatches/2), np.sin(-anglePatches/2), 0.]) ]
# Defines patchy protein potential
potentialPatchyProteinMAPK = patchyProteinMAPK(sigma, strength, patchesCoordinates1, patchesCoordinates2)
# Incorporate potential into integrator
integrator.setPairPotential(potentialPatchyProteinMAPK)

In [ ]:
# Integrate the particles, save to .xyz to produce VMD output (additional overhead)
timesteps = 5000000
stride = 250 #250 #1000
datafile  = open('../../data/vmd/patchyProteinMAPK.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(9) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            if part.type == 0:
                v0 = part.position
                v1 = v0 + 0.4*sigma*quats.rotateVec(patchesCoordinates1[0], part.orientation)
                v2 = v0 + 0.4*sigma*quats.rotateVec(patchesCoordinates1[1], part.orientation)
                datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
                if (part.state == 0):
                    datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
                    datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
                    datafile.write('type_2' + ' ' + ' '.join(map(str, v0)) + '\n')
                    datafile.write('type_2' + ' ' + ' '.join(map(str, v0)) + '\n')
                elif (part.state == 1):
                    datafile.write('type_1' + ' ' + ' '.join(map(str, v0)) + '\n')
                    datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
                    datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
                    datafile.write('type_2' + ' ' + ' '.join(map(str, v0)) + '\n')
                elif (part.state == 2):
                    datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
                    datafile.write('type_1' + ' ' + ' '.join(map(str, v0)) + '\n')
                    datafile.write('type_2' + ' ' + ' '.join(map(str, v0)) + '\n')
                    datafile.write('type_2' + ' ' + ' '.join(map(str, v2)) + '\n')
                else:
                    datafile.write('type_1' + ' ' + ' '.join(map(str, v0)) + '\n')
                    datafile.write('type_1' + ' ' + ' '.join(map(str, v0)) + '\n')
                    datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
                    datafile.write('type_2' + ' ' + ' '.join(map(str, v2)) + '\n')
            if part.type == 1:
                v0 = part.position
                if part.state == 0:
                    v1 = v0 + 0.4*sigma*quats.rotateVec(patchesCoordinates2[0], part.orientation)
                else:
                    v1 = 1 * v0
                datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
                datafile.write('type_2' + ' ' + ' '.join(map(str, v1)) + '\n')
            if part.type == 2:
                v0 = part.position
                if part.state == 0:
                    v1 = v0 + 0.4*sigma*quats.rotateVec(patchesCoordinates2[0], part.orientation)
                else:
                    v1 = 1 * v0
                datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
                datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
    integrator.integrate(partlist)
    if i%5000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyProteinMAPK(numparticles = numparticles, 
                                    outfname = "patchyProteinMAPK", 
                                    tclfname = "../../data/vmd/patchyProteinMAPK_2vmd.tcl")
print("Percentage complete: ", 100, " %")

To load the movie go to /data/vmd and run in a terminal "vmd -e patchyProteinMAPK_2vmd.tcl".

## In case VMD is not desired

In case VMD output is not desired, below we simply integrate the model and show the output directly.

In [ ]:
# Integrate particle list and print only positions of first and last particle in list
timesteps = 100
print('{:<10s}{:<15s}{:<40s}{:<40s}'.format("Iteration", "Time", "Position 1", "Position 2"))
for i in range(timesteps):
    print('{:<10d}{:<15f}{:<40s}{:<50s}'.format(i, integrator.clock, str(partlist[0].position), str(partlist[1].position)))
    print('{:<10d}{:<15f}{:<40s}{:<50s}'.format(i, integrator.clock, str(partlist[0].state), str(partlist[1].state)))

    integrator.integrate(partlist)